In [1]:
# import importlib
# import local_utils; importlib.reload(local_utils)
from local_utils import *

/opt/conda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Seed: 7961730


In [2]:
# Load Data
ids, comments, Y, test_ids, test_comments, inx2label, label2inx = load_data()
Y_wblank = np.concatenate([Y, np.expand_dims((~Y.any(axis=1)).astype(int), 1)], axis=1)

In [21]:
# bad_comments = np.extract(Y.sum(axis=1) > 0, comments)

In [3]:
def preprocess(text):
    return re.sub("\t|\n", " ", re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", " ", unidecode.unidecode(text.lower())))

comments = Parallel(n_jobs=cpu_cores)(delayed(preprocess)(text) for text in comments)
test_comments = Parallel(n_jobs=cpu_cores)(delayed(preprocess)(text) for text in test_comments)

In [4]:
# # SentencePiece

# with open('bpe/train.txt', 'w', encoding='utf-8') as file:
#     for i in range(0, len(comments)):
#         file.write(comments[i]+'\n')

# with open('bpe/test.txt', 'w', encoding='utf-8') as file:
#     for i in range(0, len(test_comments)):
#         file.write(test_comments[i]+'\n')

# spm_train --input=train.txt,test.txt --model_prefix=toxic --vocab_size=8000 --model_type=unigram --num_threads 8
# spm_export_vocab --model=toxic.model --output=toxic.vocab.txt

# spm_encode --model=toxic.model --output_format=piece < train.txt > train_piece.txt
# spm_encode --model=toxic.model --output_format=piece < test.txt > test_piece.txt

# spm_encode --model=toxic.model --output_format=id < train.txt > train_id.txt
# spm_encode --model=toxic.model --output_format=id < test.txt > test_id.txt

# with open('bpe/train_piece.txt', 'r', encoding='utf-8') as file:
#     piece_seq = [line.rstrip('\n') for line in file]
        
# with open('bpe/test_piece.txt', 'r', encoding='utf-8') as file:
#     test_piece_seq = [line.rstrip('\n') for line in file]

# with open('bpe/train_id.txt', 'r', encoding='utf-8') as file:
#     id_seq = []
#     for line in file:
#         id_seq.append([int(i) for i in line.rstrip('\n').split(' ')])    
    
# with open('bpe/test_id.txt', 'r', encoding='utf-8') as file:
#     test_id_seq = []
#     for line in file:
#         test_id_seq.append([int(i) for i in line.rstrip('\n').split(' ')])

In [5]:
num_op = 50000

import sentencepiece as spm
sp = spm.SentencePieceProcessor()
# sp.Load("bpe/bpemb/en.wiki.bpe.op"+str(num_op)+".model")
sp.Load("bpe/toxic.model")

def encode_ids(text):
    return np.array(sp.EncodeAsIds(text))+1

def encode_pieces(text):
    return sp.EncodeAsPieces(text)

id_seq = Parallel(n_jobs=cpu_cores)(delayed(encode_ids)(comment) for comment in comments)
test_id_seq = Parallel(n_jobs=cpu_cores)(delayed(encode_ids)(comment) for comment in test_comments)

piece_seq = Parallel(n_jobs=cpu_cores)(delayed(encode_pieces)(comment) for comment in comments[:100])
# test_piece_seq = Parallel(n_jobs=cpu_cores)(delayed(encode_pieces)(comment) for comment in test_comments)

In [6]:
doc_len = []
doc_ulen = []
for seq in id_seq:
    doc_len.append(len(seq))
    doc_ulen.append(len(set(seq)))
    
for seq in test_id_seq:
    doc_len.append(len(seq))
    doc_ulen.append(len(set(seq)))
    
meta = np.stack([np.array(doc_len), np.array(doc_ulen)], axis=-1)
meta_mean = meta.mean(axis=0)
meta_std = meta.std(axis=0)
meta = (meta - meta_mean)/meta_std

print(meta_mean)
print(meta_std)
print(meta_mean[0]+2*meta_std[0])
print(meta_mean[0]+3*meta_std[0])

X_meta = meta[:len(comments)]
test_X_meta = meta[len(comments):]

[94.04877612 56.04054551]
[158.14612833  59.17649288]
410.34103277846043
568.4871611075118


In [7]:
max_len = 600
# max_len = int(meta_mean[0]+3*meta_std[0])
emb_size = 50

# PAD_TOKEN = "_PAD_"
# pad_vec = np.zeros((1, emb_size))

# from gensim.models import KeyedVectors
# model = KeyedVectors.load_word2vec_format("bpe/bpemb/en.wiki.bpe.op"+str(num_op)+".d"+str(emb_size)+".w2v.bin", binary=True)
# emb_vectors = np.concatenate([pad_vec, model.syn0])
# print(emb_vectors.shape)

X = pad_sequences(id_seq, max_len)
test_X = pad_sequences(test_id_seq, max_len)

In [8]:
# Train/Valid splitting
trn_inx, val_inx = stratified_sampling(Y, 0.1, seed)

print("train: {}, valid: {}".format(len(trn_inx), len(val_inx)))
# plot_stratified_sampling(Y, trn_inx, val_inx, inx2label)

train: 143613, valid: 15958


In [9]:
# dpcnn
from keras.initializers import RandomNormal
def dpcnn(embedding_matrix=None, embedding_size=emb_size,
          maxlen=max_len, max_features=8001,
          filter_nr=64, kernel_size=3, repeat_block=2, dropout_convo=0.25,
          dense_size=256, repeat_dense=0, dropout_dense=0.25,
          l2_reg_convo=0.0, l2_reg_dense=0.0, use_prelu=True,
          trainable_embedding=True, use_batch_norm=True):
    """
    Note:
        Implementation of http://ai.tencent.com/ailab/media/publications/ACL3-Brady.pdf
        post activation is used instead of pre-activation, could be worth exploring
    """

    input_text = Input(shape=(maxlen,))
    if embedding_matrix is not None:
        embedding = Embedding(max_features, embedding_size, weights=[embedding_matrix], trainable=trainable_embedding)(input_text)
    else:
        embedding = Embedding(max_features, embedding_size)(input_text)

    embedding = SpatialDropout1D(0.15)(embedding)
    
    x = _convolutional_block(filter_nr, kernel_size, use_batch_norm, use_prelu, dropout_convo, l2_reg_convo)(embedding)
    x = _convolutional_block(filter_nr, kernel_size, use_batch_norm, use_prelu, dropout_convo, l2_reg_convo)(x)
    if embedding_size == filter_nr:
        x = add([embedding, x])
    else:
        embedding_resized = _shape_matching_layer(filter_nr, use_prelu, dropout_convo, l2_reg_convo)(embedding)
        x = add([embedding_resized, x])
    for _ in range(repeat_block):
        x = _dpcnn_block(filter_nr, kernel_size, use_batch_norm, use_prelu, dropout_convo, l2_reg_convo)(x)

    x = GlobalMaxPool1D()(x)
    for _ in range(repeat_dense):
        x = _dense_block(dense_size, use_batch_norm, use_prelu, dropout_dense, l2_reg_dense)(x)
    predictions = Dense(6, activation="sigmoid")(x)

    model = Model(inputs=input_text, outputs=predictions)
    return model

def _convolutional_block(filter_nr, kernel_size, use_batch_norm, use_prelu, dropout, l2_reg):
    def f(x):
        x = Conv1D(filter_nr, kernel_size=kernel_size, padding='same', activation='linear',
                   kernel_initializer=RandomNormal(mean=0.0, stddev=0.001),
                   kernel_regularizer=regularizers.l2(l2_reg))(x)
        x = _bn_relu_dropout_block(use_batch_norm, use_prelu, dropout)(x)
        return x

    return f

def _bn_relu_dropout_block(use_batch_norm, use_prelu, dropout):
    def f(x):
        if use_batch_norm:
            x = BatchNormalization()(x)
        if use_prelu:
            x = PReLU()(x)
        else:
            x = Activation('relu')(x)
        x = Dropout(dropout)(x)
        return x

    return f

def _shape_matching_layer(filter_nr, use_prelu, dropout, l2_reg):
    def f(x):
        x = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear',
                   kernel_initializer=RandomNormal(mean=0.0, stddev=0.001),
                   kernel_regularizer=regularizers.l2(l2_reg))(x)
        if use_prelu:
            x = PReLU()(x)
        else:
            x = Activation('relu')(x)
        x = Dropout(dropout)(x)
        return x

    return f

def _dpcnn_block(filter_nr, kernel_size, use_batch_norm, use_prelu, dropout, l2_reg):
    def f(x):
        x = MaxPooling1D(pool_size=3, strides=2)(x)
        main = _convolutional_block(filter_nr, kernel_size, use_batch_norm, use_prelu, dropout, l2_reg)(x)
        main = _convolutional_block(filter_nr, kernel_size, use_batch_norm, use_prelu, dropout, l2_reg)(main)
        x = add([main, x])
        return x

    return f

def _dense_block(dense_size, use_batch_norm, use_prelu, dropout, l2_reg):
    def f(x):
        x = Dense(dense_size, activation='linear',
                  kernel_regularizer=regularizers.l2(l2_reg))(x)
        x = _bn_relu_dropout_block(use_batch_norm, use_prelu, dropout)(x)
        return x

    return f

In [10]:
# Model description
# model_name = 'bpe_op'+str(num_op)+'_d'+str(emb_size)+'_model'
model_name = 'bpe_8000_model'
def getDMModel(num_words, emb_size=emb_size, input_shape=X.shape[1], classes=Y.shape[1]):

    x_input = Input(shape=(input_shape,))
    
    emb = Embedding(num_words, emb_size, trainable=True, input_length=input_shape, 
#                     embeddings_regularizer=l1_l2(l2=1e-6),
                    name='embs')(x_input)
    emb = Activation('tanh')(emb)
    emb = SpatialDropout1D(0.3)(emb)
        
    rnn1 = Bidirectional(CuDNNGRU(100, return_sequences=True))(emb)
    rnn1 = SpatialDropout1D(0.3)(rnn1)
    rnn2 = Bidirectional(CuDNNGRU(100, return_sequences=True))(rnn1)
    rnn2 = SpatialDropout1D(0.3)(rnn2)
    x = concatenate([emb, rnn1, rnn2])
    x = AttentionWeightedAverage()(x)

    x_output = Dense(classes, activation='sigmoid')(x)
    return Model(inputs=x_input, outputs=x_output)

model = getDMModel(num_words=8001)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 600)          0                                            
__________________________________________________________________________________________________
embs (Embedding)                (None, 600, 20)      160020      input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 600, 20)      0           embs[0][0]                       
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 600, 20)      0           activation_1[0][0]               
__________________________________________________________________________________________________
bidirectio

In [11]:
model_loss_checkpoint = ModelCheckpoint(models_dir+model_name+'_loss.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
model_auc_checkpoint = ModelCheckpoint(models_dir+model_name+'_auc.h5', monitor='val_auc_roc', mode='max', verbose=1, save_best_only=True)

def lr_change(i, lr): 
    if (i == 0): return lr
#     if (i == 3): return 0.001
    return lr*0.85

lr_schedule = LearningRateScheduler(lr_change, verbose=1)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=1, min_lr=0.0001, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir='logs', write_graph=False)
roc_auc_callback = ROCAUCCallback(X[val_inx], Y[val_inx], 1024)

In [12]:
batch_size=512
trn_seq = StratifiedFeatureSequence(X[trn_inx], Y[trn_inx], batch_size)
# trn_seq = FeatureSequence(X[trn_inx], X_meta[trn_inx], Y[trn_inx], batch_size, shuffle=True)
val_seq = FeatureSequence(X[val_inx], X_meta[val_inx], Y[val_inx], batch_size)

In [13]:
def getWeights(Y, mu=0.5):
    return np.array([w for w in np.log(mu*Y.shape[0]/Y.sum(axis=0))])
weights = getWeights(Y)

In [14]:
def roc_auc_loss(y_true, y_pred):
    """ ROC AUC Score.
    Approximates the Area Under Curve score, using approximation based on
    the Wilcoxon-Mann-Whitney U statistic.
    Yan, L., Dodier, R., Mozer, M. C., & Wolniewicz, R. (2003).
    Optimizing Classifier Performance via an Approximation to the Wilcoxon-Mann-Whitney Statistic.
    Measures overall performance for a full range of threshold levels.
    Arguments:
        y_pred: `Tensor`. Predicted values.
        y_true: `Tensor` . Targets (labels), a probability distribution.
    """
    with tf.name_scope("RocAucScore"):

        pos = tf.boolean_mask(y_pred, tf.cast(y_true, tf.bool))
        neg = tf.boolean_mask(y_pred, ~tf.cast(y_true, tf.bool))

        pos = tf.expand_dims(pos, 0)
        neg = tf.expand_dims(neg, 1)

        # original paper suggests performance is robust to exact parameter choice
        gamma = 0.2
        p     = 3 # 3

        difference = tf.zeros_like(pos * neg) + pos - neg - gamma

        masked = tf.boolean_mask(difference, difference < 0.0)

        return tf.reduce_sum(tf.pow(-masked, p))

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(0.003))
# model.compile(loss=roc_auc_loss, optimizer=optimizers.RMSprop(0.003, clipnorm=1.0))

In [15]:
epochs=15
model.fit_generator(
    generator=trn_seq, steps_per_epoch=len(trn_seq),
    validation_data=val_seq, validation_steps=len(val_seq),
    initial_epoch=0, epochs=epochs, shuffle=False, verbose=1,
#     class_weight=weights,
    callbacks=[model_loss_checkpoint, lr_schedule, roc_auc_callback],
    use_multiprocessing=False, workers=cpu_cores, max_queue_size=4*cpu_cores)

/opt/conda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15

Epoch 00001: LearningRateScheduler reducing learning rate to 0.003000000026077032.
281/281 [==============================] - 53s 188ms/step - loss: 593.5352 - val_loss: 176.3427

Epoch 00001: val_loss improved from inf to 176.34269, saving model to models/bpe_8000_model_loss.h5
roc-auc_val: 0.97140664                                                                                                    
Epoch 2/15

Epoch 00002: LearningRateScheduler reducing learning rate to 0.002550000022165477.
281/281 [==============================] - 52s 185ms/step - loss: 191.1583 - val_loss: 153.8260

Epoch 00002: val_loss improved from 176.34269 to 153.82601, saving model to models/bpe_8000_model_loss.h5
roc-auc_val: 0.97421041                                                                                                    
Epoch 3/15

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0021675000782124696.
281/281 [==============================] - 52s 186ms/step - loss: 

In [16]:
del model
model = load_model(models_dir+model_name+'_loss.h5', compile=True, 
                   custom_objects={'Attention':Attention, 'AttentionWeightedAverage':AttentionWeightedAverage, 
                                   'focal_loss':focal_loss, 'roc_auc_loss':roc_auc_loss})

/opt/conda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [17]:
Y_trn_pred = model.predict(X[trn_inx], batch_size=1024, verbose=0)
losses = compute_losses(Y[trn_inx], Y_trn_pred, eps=1e-5)

trn_loss = sum(losses)/len(losses)
trn_auc = metrics.roc_auc_score(Y[trn_inx], Y_trn_pred)

print()
print("avg_loss: {}".format(trn_loss))
print("ROC AUC: {}".format(trn_auc))


avg_loss: 0.22226397930351885
ROC AUC: 0.9890397796329746


In [18]:
Y_val_pred = model.predict(X[val_inx], batch_size=512, verbose=0)
losses = compute_losses(Y[val_inx], Y_val_pred, eps=1e-5)

val_loss = sum(losses)/len(losses)
val_auc = metrics.roc_auc_score(Y[val_inx], Y_val_pred)

print()
print("avg_loss: {}".format(val_loss))
print("ROC AUC: {}".format(val_auc))


avg_loss: 0.22331672980188522
ROC AUC: 0.9813257089824635


In [19]:
submission_name = 'bpe_submission_'+str(round(val_loss, 5))+'_'+str(round(val_auc, 5))+'.csv'

sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')
test_pred = model.predict(test_X, batch_size=1024, verbose=1)
sample_submission[inx2label] = test_pred
sample_submission.to_csv(results_dir+submission_name, index=False)

FileLink(results_dir+submission_name)

153164/153164 [==============================] - 11s 75us/step


/src/DL/skolbachev/toxic/results/bpe_submission_0.22332_0.98133.csv

In [20]:
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument, KeyedVectors

# def build_tagged_docs(docs):
#     return [TaggedDocument(tokens, [tag]) for tag, tokens in enumerate(docs)]

# d2v_corpus = build_tagged_docs(text_analyzer.transform_to_tokens(piece_seq)+text_analyzer.transform_to_tokens(test_piece_seq))
# print(len(d2v_corpus))

# d2v_model = Doc2Vec(dm=0, dbow_words=1, size=300, window=12, min_count=0, iter=30, workers=cpu_cores)

# %time d2v_model.build_vocab(d2v_corpus)
# print(len(d2v_model.wv.vocab))

# %time d2v_model.train(d2v_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.iter)
# d2v_model.save('bpe/bpe_d2v_model')

# from gensim.models.doc2vec import Doc2Vec
# d2v_model = Doc2Vec.load('bpe/bpe_d2v_model')

# emb_matrix = [np.zeros(300)]
# for w in text_analyzer.emb_words[1:]:
#     emb_matrix.append(d2v_model.wv[w])
# #     emb_matrix.append(d2v_model.wv[w]/np.linalg.norm(d2v_model.wv[w]))
    
# emb_matrix = np.stack(emb_matrix)